In [ ]:
from datetime import datetime
from IPython.display import HTML
import pandas as pd
import arcgis
import getpass

### Authenticate with Web GIS

In [ ]:
portals_dict = {
    "esrifederal_gis": r"https://esrifederal.maps.arcgis.com",
    "natgov_gis": r"http://esri-natgov105.eastus.cloudapp.azure.com/arcgis",
    "dot_gis": r"http://dot.esri.com/portal",
    "dev_gis": r"http://govdev.eastus.cloudapp.azure.com/arcgis",
    "local_gis": r"https://anieto.esri.com/arcgis"
}

In [ ]:
gis_url = portals_dict["esrifederal_gis"]

In [ ]:
if gis_url == portals_dict["esrifederal_gis"]:
    gis_app_id = r"eIIyPrDiF9hsZ6cO"
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, client_id=gis_app_id)
    print("Successfully logged in as: " + gis.properties.user.username)
else:
    gis_username = getpass.getpass(prompt="Username: ")
    gis_pw = getpass.getpass(prompt="Password: ")
    print("Attempting to log in to '{0}'...".format(gis_url))
    gis = arcgis.gis.GIS(gis_url, gis_username, gis_pw, verify_cert=False)
    print("Successfully logged in as: " + gis.properties.user.username)

In [ ]:
import arcgis.network as network

In [ ]:
service_area_url = gis.properties.helperServices.serviceArea.url
service_area_url

In [ ]:
sa_layer = network.ServiceAreaLayer(service_area_url, gis=gis)

## Create the facilities layer

In [ ]:
fire_station_1_coord = '139.546910,35.695729'
fire_station_2_coord = '139.673726,35.697988'

from arcgis.features import Feature, FeatureSet

f1 = Feature(geometry={'x':float(fire_station_1_coord.split(',')[0]),
                      'y':float(fire_station_1_coord.split(',')[1])})

f2 = Feature(geometry={'x':float(fire_station_2_coord.split(',')[0]),
                      'y':float(fire_station_2_coord.split(',')[1])})

fire_station_fset = FeatureSet([f1,f2], geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

fire_station_fset

In [ ]:
map1 = gis.map('Tokyo', zoomlevel=12)
map1

In [ ]:
fire_truck_symbol = {"type":"esriPMS",
                     "url":"http://static.arcgis.com/images/Symbols/SafetyHealth/FireTruck.png",
                     "contentType": "image/png", "width":20, "height":20}

map1.draw(fire_station_fset, symbol=fire_truck_symbol)

## Compute service area

In [ ]:
travel_modes = sa_layer.retrieve_travel_modes()
truck_mode = [t for t in travel_modes['supportedTravelModes'] if t['name'] == 'Trucking Time'][0]

result = sa_layer.solve_service_area(fire_station_fset, default_breaks=[5,10,15], 
                                     travel_direction='esriNATravelDirectionFromFacility',
                                     travel_mode=truck_mode)

In [ ]:
result.keys()

In [ ]:
result['saPolygons'].keys()

In [ ]:
poly_feat_list = []
for polygon_dict in result['saPolygons']['features']:
    f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
    poly_feat_list.append(f1)
poly_feat_list

In [ ]:
service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])

In [ ]:
service_area_fset.df

## Visualize the service area on the map

In [ ]:
colors = {5: [0, 128, 0, 90], 
          10: [255, 255, 0, 90], 
          15: [255, 0, 0, 90]}

fill_symbol = {"type": "esriSFS","style": "esriSFSSolid",
               "color": [115,76,0,255]}

In [ ]:
for service_area in service_area_fset.features:
    
    #set color based on drive time
    fill_symbol['color'] = colors[service_area.attributes['ToBreak']]
    
    #set popup
    popup={"title": "Service area", 
            "content": "{} minutes".format(service_area.attributes['ToBreak'])}
    
    map1.draw(service_area.geometry, symbol=fill_symbol, popup=popup)

## Constructing service areas for different times of the day

In [ ]:
times = [datetime(2017, 6, 10, h).timestamp() * 1000 
         for h in [6, 10, 14, 18, 22]]

# fire station location
fire_station = '-118.245847, 34.038608'

#loop through each time of the day and compute the service area
sa_results = []
for daytime in times:
    result = sa_layer.solve_service_area(facilities=fire_station, default_breaks=[5,10,15], 
                                         travel_direction='esriNATravelDirectionFromFacility',
                                         time_of_day=daytime, time_of_day_is_utc=False)
    sa_results.append(result)
sa_results

In [ ]:
LA_fset_list=[]
for result in sa_results:
    poly_feat_list = []
    for polygon_dict in result['saPolygons']['features']:
        f1 = Feature(polygon_dict['geometry'], polygon_dict['attributes'])
        poly_feat_list.append(f1)
        
    service_area_fset = FeatureSet(poly_feat_list, 
                         geometry_type=result['saPolygons']['geometryType'],
                         spatial_reference= result['saPolygons']['spatialReference'])
    
    LA_fset_list.append(service_area_fset)
LA_fset_list    

In [ ]:
map2= gis.map("Los Angeles, CA")
map2

In [ ]:
import time
map2.clear_graphics()

times = ['6 am', '10 am', '2 pm', '6 pm', '10 pm']
j=0
time.sleep(2)

for fset in LA_fset_list:
    print(times[j])
    map2.draw(fset)
    j+=1
    time.sleep(1)